<a href="https://colab.research.google.com/github/meetreks/SaiSpark/blob/master/Spark_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark


In [ ]:
!nvidia-smi

In [ ]:
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar

In [ ]:
!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar
!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar

In [ ]:
!ls
!pwd

In [14]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [17]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/cudf-0.9.2-cuda10-1.jar,/content/xgboost4j_2.x-1.0.0-Beta5.jar,/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar pyspark-shell'

In [21]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
sc = SparkSession.builder.master("local[*]").getOrCreate()
#sc.conf.set("spark.executor.memory", "2g")
#sc.conf.set("spark.driver.memory", "2g")
sc.sparkContext.addPyFile('/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar')

In [22]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from ml.dmlc.xgboost4j.scala.spark.rapids import GpuDataReader
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
import pandas as pd

In [23]:
from sklearn.datasets import fetch_openml
covtyp = fetch_openml(name='covertype', version=4)

In [24]:
cov_df = pd.DataFrame(data= np.c_[covtyp['data'], covtyp['target']],
                     columns= covtyp['feature_names'] + ['target'])

In [25]:
cov_df.memory_usage(index=True).sum()

255645408

In [26]:
for cols in cov_df.columns:
  cov_df[cols] = pd.to_numeric(cov_df[cols])

In [27]:
cov_df['target'] = cov_df['target']-1

In [28]:
train_df=cov_df.sample(frac=0.8,random_state=10) 
test_df=cov_df.drop(train_df.index)

In [ ]:
train_df.to_parquet(fname='covtype_train.parquet',compression='snappy', index=False)
test_df.to_parquet(fname='covtype_test.parquet',compression='snappy', index=False)

In [30]:
import pyarrow.parquet as pq

In [ ]:
pq.read_table('covtype_train.parquet')

In [33]:
train_data = GpuDataReader(sc).format('parquet').load('covtype_train.parquet')
test_data = GpuDataReader(sc).format('parquet').load('covtype_test.parquet')

In [35]:
pq_file=pq.read_table('covtype_train.parquet')
label="target"
features = [ x for x in pq_file.column_names if x != label ]

In [ ]:
features

In [ ]:
import time
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 8, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'numWorkers': 1,
    'verbosity': 1
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)
#YOU WILL GET AN ERROR EXECUTING THE ABOVE STATEMENT BECAUSE 
#XGBOOST does not support Spark 3.0.0. Please use Spark 2.4.3.

In [4]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [13]:
!ls

covtype_test.parquet	       spark-2.4.7-bin-hadoop2.7.tgz.1
covtype_train.parquet	       spark-2.4.7.tgz
cudf-0.9.2-cuda10-1.jar        spark-2.4.7.tgz.1
cudf-0.9.2-cuda10-1.jar.1      xgboost4j_2.x-1.0.0-Beta5.jar
sample_data		       xgboost4j_2.x-1.0.0-Beta5.jar.1
spark-2.4.7-bin-hadoop2.7      xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark-2.4.7-bin-hadoop2.7.tgz  xgboost4j-spark_2.x-1.0.0-Beta5.jar.1


In [14]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/cudf-0.9.2-cuda10-1.jar,/content/xgboost4j_2.x-1.0.0-Beta5.jar,/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar pyspark-shell'

In [15]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
#spark.conf.set("spark.executor.memory", "2g")
#spark.conf.set("spark.driver.memory", "2g")
spark.sparkContext.addPyFile('/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar')

In [16]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from ml.dmlc.xgboost4j.scala.spark.rapids import GpuDataReader
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
covtyp = fetch_openml(name='covertype', version=4)

cov_df = pd.DataFrame(data= np.c_[covtyp['data'], covtyp['target']],
                     columns= covtyp['feature_names'] + ['target'])

for cols in cov_df.columns:
  cov_df[cols] = pd.to_numeric(cov_df[cols])

cov_df['target'] = cov_df['target']-1
train_df=cov_df.sample(frac=0.8,random_state=10) 
test_df=cov_df.drop(train_df.index)

In [17]:
train_data = GpuDataReader(spark).format('parquet').load('covtype_train.parquet')
test_data = GpuDataReader(spark).format('parquet').load('covtype_test.parquet')

In [19]:
import pyarrow.parquet as pq
pq_file=pq.read_table('covtype_train.parquet')
label="target"
features = [ x for x in pq_file.column_names if x != label ]


In [20]:
import time
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 8, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'numWorkers': 1,
    'verbosity': 1
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

In [ ]:
#Stil will get a problem. In you want to resolve - diconnect and connect

In [21]:
start_time = time.time()

model=classifier.fit(train_data)

print("GPU Training Time: %s seconds" % (str(time.time() - start_time)))

GPU Training Time: 27.310513496398926 seconds


In [ ]:
!nvidia-smi
model.write().overwrite().save('/content/model/')
loaded_model = XGBoostClassificationModel().load('/content/model/')
result=loaded_model.transform(test_data)
result.show()
MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result)

In [6]:
!rm -rf /content/spark-3.0.1-bin-hadoop3.2

In [ ]:
!ls

In [11]:
!rm -rf /content/spark-3.0.1-bin-hadoop3.2.tgz.1

In [ ]:
###Now it works maddy - you are a star